In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers datasets
!pip install hnswlib

In [2]:
import torch
from transformers import pipeline, set_seed
import torch.nn as nn
import hnswlib
import os
import timeit

/home/martin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class HNSWIndexEmbedding():
    def __init__(self, weight, k, M=32, ef=100, ef_construction=100, index_file=None):
        self.k, self.vocab_size, self.dim = k, weight.shape[0], weight.shape[1]
        self.index = hnswlib.Index(space='ip', dim=self.dim)
        self.index.init_index(max_elements=self.vocab_size, M=M, ef_construction=ef_construction, random_seed=42)

        index_path = f"{index_file.replace("/", "-")}-{M}-{ef_construction}.index"
        if index_file is None or not os.path.exists(index_path):
            self.index.add_items(weight.numpy())
            if index_file is not None:
                self.index.save_index(index_path)
        else:
            print(f"Loading index from file: {index_path}")
            self.index.load_index(index_path)
        self.index.set_ef(ef)

    def forward(self, x):
        indices, distances = self.index.knn_query(x.detach().cpu().numpy(), k=self.k)
        return torch.from_numpy(1 - distances).to(torch.float32).to(x.device), torch.from_numpy(indices).to(torch.int64).to(x.device)


class HNSWLogitsEmbedding(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer

    def forward(self, x):
        x_flat = x.view(-1, x.shape[-1])
        distances, indices = self.layer.forward(x_flat)
   
        logits = torch.full((x_flat.shape[0], self.layer.vocab_size), float("-inf"), dtype=torch.float32, device=x.device)
        logits.scatter_(-1, indices, distances)
        return logits.view((x.shape[0], x.shape[1], self.layer.vocab_size))

In [10]:
model_name = "meta-llama/Llama-3.2-1B" # "gpt2" # "meta-llama/Llama-3.2-1B" # "meta-llama/Llama-3.2-3B"
generator = pipeline('text-generation', model=model_name, device="cpu")
generator_ref = pipeline('text-generation', model=model_name, device="cpu")
eos_token_id = generator.tokenizer.eos_token_id
set_seed(42)

Device set to use cpu
Device set to use cpu


In [11]:
weight = generator.model.lm_head.weight.detach().clone()
generator.model.lm_head = HNSWLogitsEmbedding(HNSWIndexEmbedding(weight, k=50, ef=100, ef_construction=50, index_file=model_name))

In [6]:
max_new_tokens=8
num_repeat=2

In [7]:
time_vector = timeit.repeat(lambda: generator("Hello, I'm a language model,", max_new_tokens=max_new_tokens, num_return_sequences=1, pad_token_id=eos_token_id), number=1, repeat=num_repeat)
print(time_vector)

[0.8324419839991606, 0.30504160699638305]


In [8]:
time_ref = timeit.repeat(lambda: generator_ref("Hello, I'm a language model,", max_new_tokens=max_new_tokens, num_return_sequences=1, pad_token_id=eos_token_id), number=1, repeat=num_repeat)
print(time_ref)

[0.34858985699975165, 0.3669258139998419]


In [9]:
print(f"Vec: {min(time_vector):.2f}")
print(f"Ref: {min(time_ref):.2f}")
print(f"Speedup: {min(time_ref) / min(time_vector):.4f}x")

Vec: 0.31
Ref: 0.35
Speedup: 1.1428x


In [15]:
model_runs = {
    # "gpt2": [32, 64, 128, 256, 512],
    # "meta-llama/Llama-3.2-1B": [32, 64, 128, 256],
    # "meta-llama/Llama-3.2-3B": [32, 64, 128],
}

table = []
for model_name, num_tokens  in model_runs.items():
    print(f"Running {model_name}")
    generator = pipeline('text-generation', model=model_name, device="cpu")
    generator_ref = pipeline('text-generation', model=model_name, device="cpu")
    eos_token_id = generator.tokenizer.eos_token_id
    
    weight = generator.model.lm_head.weight.detach().clone()
    generator.model.lm_head = HNSWLogitsEmbedding(HNSWIndexEmbedding(weight, k=50, ef=150, ef_construction=500, index_file=model_name))
    
    for num_new_token in num_tokens:
        num_repeat = 2 if num_new_token <= 128 else 2
        time_vector = timeit.repeat(lambda: generator("Hello, I'm a language model,", max_new_tokens=num_new_token, num_return_sequences=1, pad_token_id=eos_token_id), number=1, repeat=num_repeat)
        time_ref = timeit.repeat(lambda: generator_ref("Hello, I'm a language model,", max_new_tokens=num_new_token, num_return_sequences=1, pad_token_id=eos_token_id), number=1, repeat=num_repeat)
        speedup = min(time_ref) / min(time_vector)
        print(f"{model_name}/{num_new_token}: vec={min(time_vector):.2f} ref={min(time_ref):.2f} speedup={speedup:.4f}x, vec={time_vector}, ref={time_ref}")
        table.append([model_name, num_new_token, min(time_vector), min(time_ref), speedup])

print("\n")
print("| Model | Tokens | Vec | Ref | Speedup |")
print("\n".join(["| " + " | ".join([str(round(x, 2)) if isinstance(x, float) else str(x) for x in row]) + " |" for row in table]))

Running meta-llama/Llama-3.2-1B


Device set to use cpu
Device set to use cpu


Loading index from file: meta-llama-Llama-3.2-1B-32-500.index


meta-llama/Llama-3.2-1B/32: vec=9.12 ref=11.01 speedup=1.2070x, vec=[9.554434083998785, 9.122559716000978], ref=[11.010991518996889, 11.39642112999718]
meta-llama/Llama-3.2-1B/64: vec=18.07 ref=19.27 speedup=1.0669x, vec=[18.06607265400089, 18.40356360399892], ref=[19.273854510996898, 23.17641081399779]
meta-llama/Llama-3.2-1B/96: vec=28.17 ref=34.87 speedup=1.2376x, vec=[28.174450714999693, 28.341151351996814], ref=[35.1987951800038, 34.86783388800541]


| Model | Tokens | Vec | Ref | Speedup |
| meta-llama/Llama-3.2-1B | 32 | 9.12 | 11.01 | 1.21 |
| meta-llama/Llama-3.2-1B | 64 | 18.07 | 19.27 | 1.07 |
| meta-llama/Llama-3.2-1B | 96 | 28.17 | 34.87 | 1.24 |


In [11]:
"""
Running gpt2
gpt2/32: vec=0.94 ref=1.31 speedup=1.3876x, vec=[0.9552368999975442, 0.9865360850017169, 0.9439662329968996, 0.9508174799993867], ref=[1.3306904579985712, 1.3728614949977782, 1.4314335310009483, 1.3098488989999169]
gpt2/64: vec=1.83 ref=2.55 speedup=1.3906x, vec=[1.8967858869982592, 1.8778003670013277, 1.9814240359992255, 1.8336702579981647], ref=[2.549973429999227, 2.624112575002073, 2.737567322998075, 2.7280078310031968]
gpt2/128: vec=3.71 ref=5.14 speedup=1.3861x, vec=[3.8986998920008773, 4.046735912001168, 3.7104772309976397, 3.7446656729989627], ref=[5.281697868002084, 5.180498829999124, 5.177037355999346, 5.143146140999306]
gpt2/256: vec=7.83 ref=10.34 speedup=1.3209x, vec=[7.828270706999319, 8.285325303000718], ref=[11.455289707999327, 10.340517969998473]
gpt2/512: vec=13.70 ref=23.07 speedup=1.6836x, vec=[13.702081597999495, 16.435789234998083], ref=[23.220302097997774, 23.068521338998835]

Running meta-llama/Llama-3.2-1B
meta-llama/Llama-3.2-1B/32: vec=8.84 ref=10.68 speedup=1.2085x, vec=[8.838001199001155, 9.0320096989999, 9.280967007998697, 9.012281543000427], ref=[11.01587938800003, 11.516645974999847, 10.994546160000027, 10.681135528000596]
meta-llama/Llama-3.2-1B/64: vec=17.84 ref=22.26 speedup=1.2476x, vec=[17.97946036600115, 17.843435440001485, 17.947532672998932, 18.00394085299922], ref=[22.26216995700088, 22.848842208997667, 22.401243424999848, 22.608057655997982]
meta-llama/Llama-3.2-1B/128: vec=35.52 ref=17.49 speedup=0.4922x, vec=[35.52406267799961, 36.604303918997175, 36.42194557499897, 36.864600595999946], ref=[44.70566329000212, 45.02082253399931, 17.48606040300001, 24.59050794599898]
meta-llama/Llama-3.2-1B/256: vec=71.64 ref=88.00 speedup=1.2282x, vec=[71.64407563499844, 72.9635706500012], ref=[89.54998209900077, 87.99650553100219]

Running meta-llama/Llama-3.2-3B
meta-llama/Llama-3.2-3B/32: vec=26.15 ref=29.65 speedup=1.1340x, vec=[40.33591957099998, 26.148481983000238, 26.605732730000454, 26.601163373001327], ref=[49.162926258999505, 29.65189822799948, 31.952300110002398, 29.938785411002755]
meta-llama/Llama-3.2-3B/64: vec=49.75 ref=55.58 speedup=1.1172x, vec=[53.411602232001314, 52.17697993500042, 49.7500251929996, 50.23720671200135], ref=[55.581409103000624, 56.02223514900106, 55.895371186001285, 57.672117275000346]
meta-llama/Llama-3.2-3B/128: vec=99.28 ref=113.26 speedup=1.1408x, vec=[113.98015950899935, 99.28254773600202, 99.7601692399985, 99.87558131100013], ref=[119.71570586400048, 116.35653701100091, 113.26158309300081, 113.9068923900013]
"""


'\nRunning gpt2\ngpt2/32: vec=0.94 ref=1.31 speedup=1.3876x, vec=[0.9552368999975442, 0.9865360850017169, 0.9439662329968996, 0.9508174799993867], ref=[1.3306904579985712, 1.3728614949977782, 1.4314335310009483, 1.3098488989999169]\ngpt2/64: vec=1.83 ref=2.55 speedup=1.3906x, vec=[1.8967858869982592, 1.8778003670013277, 1.9814240359992255, 1.8336702579981647], ref=[2.549973429999227, 2.624112575002073, 2.737567322998075, 2.7280078310031968]\ngpt2/128: vec=3.71 ref=5.14 speedup=1.3861x, vec=[3.8986998920008773, 4.046735912001168, 3.7104772309976397, 3.7446656729989627], ref=[5.281697868002084, 5.180498829999124, 5.177037355999346, 5.143146140999306]\ngpt2/256: vec=7.83 ref=10.34 speedup=1.3209x, vec=[7.828270706999319, 8.285325303000718], ref=[11.455289707999327, 10.340517969998473]\ngpt2/512: vec=13.70 ref=23.07 speedup=1.6836x, vec=[13.702081597999495, 16.435789234998083], ref=[23.220302097997774, 23.068521338998835]\n\nRunning meta-llama/Llama-3.2-1B\nmeta-llama/Llama-3.2-1B/32: vec=